In [1]:
%reload_ext autoreload
%autoreload 2
import os
from typing import Any
from functools import partial

import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp

import flax.linen as nn
from flax.training import train_state
import optax

import tensorflow as tf
import tensorflow_datasets as tfds

from ml_collections import ConfigDict

%cd /hildafs/projects/phy230056p/junzhez/AI/maskgit/gmmg
%ls

# Import custom modules
from src.layer import get_norm_layer, tensorflow_style_avg_pooling, upsample, dsample
from src.loss import squared_euclidean_distance, entropy_loss, l1_loss, l2_loss
from src.tokenizer import VQVAE

# Directory to save images
data_dir = '/hildafs/projects/phy230056p/junzhez/data/'
data_name = 'mnist'
save_dir = os.path.join(data_dir, data_name)
os.makedirs(save_dir, exist_ok=True)

# Load the dataset
dataset_name = 'mnist'
split = 'train[:80%]'  # Using a small subset for demonstration purposes
dataset, info = tfds.load(dataset_name, data_dir=save_dir, split=split, with_info=True)
# Preprocess the dataset

def preprocess_data(sample):
    image = sample['image']
    image = tf.cast(image, tf.float32) / 255.0
    return {'image': image}
train_data = dataset.map(preprocess_data)

# Define model configuration
config = ml_collections.ConfigDict()
config.vqvae = ml_collections.ConfigDict()
config.vqvae.filters = 64
config.vqvae.num_res_blocks = 2
config.vqvae.channel_multipliers = [1, 2, 4]
config.vqvae.embedding_dim = 256
config.vqvae.conv_downsample = True
config.vqvae.norm_type = "BN"
config.vqvae.activation_fn = "relu"
config.vqvae.codebook_size = 1024
config.vqvae.commitment_cost = 0.25
config.vqvae.entropy_loss_ratio = 0.0
config.vqvae.entropy_loss_type = "softmax"
config.vqvae.entropy_temperature = 1.0


2024-05-29 01:22:01.453181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/hildafs/projects/phy230056p/junzhez/AI/maskgit/gmmg
config/       nets.ipynb  test1.ipynb        train_token.ipynb
layers.ipynb  src/        tf_datasets.ipynb


/hildafs/projects/phy230056p/junzhez/.conda/envs/AI_torch/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
2024-05-29 01:22:03.049588: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
# Define the training step
@partial(jax.jit, static_argnums=(0,))
def train_step(state, batch):
    def loss_fn(params):
        model = VQVAE(config=config, train=True)
        recon_images = model.apply({'params': params}, batch)
        recon_loss = l2_loss(batch['image'], recon_images)
        return recon_loss, recon_images

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, recon_images), grads = grad_fn(state.params)
    new_state = state.apply_gradients(grads=grads)
    return new_state, loss

# Define training state
class TrainState(train_state.TrainState):
    batch_stats: dict

# Initialize the model
rng = jax.random.PRNGKey(0)
dummy_input = jnp.ones((1, 28, 28, 1), dtype=jnp.float32)
model = VQVAE(config=config, train=True)
params = model.init(rng, {'image': dummy_input})['params']

# Initialize the training state
tx = optax.adam(learning_rate=1e-4)
state = TrainState.create(apply_fn=model.apply, params=params, tx=tx, batch_stats={})

# Training loop
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    for batch in tfds.as_numpy(train_data.batch(batch_size)):
        state, loss = train_step(state, batch)
    print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

print("Training complete.")

2024-05-29 01:22:13.004810: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


ValueError: Non-hashable static arguments are not supported. An error occurred while trying to hash an object of type <class '__main__.TrainState'>, TrainState(step=0, apply_fn=<bound method Module.apply of VQVAE(
    # attributes
    config = vqvae:
      activation_fn: relu
      channel_multipliers:
      - 1
      - 2
      - 4
      codebook_size: 1024
      commitment_cost: 0.25
      conv_downsample: true
      embedding_dim: 256
      entropy_loss_ratio: 0.0
      entropy_loss_type: softmax
      entropy_temperature: 1.0
      filters: 64
      norm_type: BN
      num_res_blocks: 2
    
    train = True
    dtype = float32
    activation_fn = relu
)>, params={'encoder': {'Conv_0': {'kernel': Array([[[[-2.25795031e-01,  1.98774144e-01,  5.51937342e-01,
          -3.54302496e-01,  1.64177362e-02,  1.45718530e-01,
          -2.99644142e-01, -1.29737526e-01,  3.35941553e-01,
           7.56987482e-02, -7.53340349e-02, -4.42623019e-01,
          -1.38274178e-01,  3.33632141e-01,  2.